# Module 2 Final Project Submission
* Team Name: M.I.A. Paper Pats (Mia Iseman and Pat Fuller)  
* Repository URL:
* Presentation URL: 

# Goal: 
## Deliver insights to help Lou Taylor decide on which musicians to manage or book for residences in Vegas. 


## Process
1. Understand the business needs
2. Understand the data
3. Prepare the data
4. Model
5. Evaluate
6. Deploy

## Business Understanding 

Lou Taylor, one of the most successful managers for musicians, boasts an all-star roster of clients, including Steven Tyler, Gwen Stefani, Florida Georgia Line and J Lo's Vegas residency. 

She has hired us to deliver insights about several factors that might help her decide on which artists to manage or book for residences in Vegas. As a jumping off point, we will answer the following questions: 
1. Are self-released albums rated differently than the wider population?
2. Are "best new music" albums more prevelant in a certain genre?  BONFERRONI IF MULTIPLE CHECK
3. Of the highest-rated labels, does on label have more "best new music" albums than the others(could do pop or could to other top ten?
4. Does one reviewer tend to review more harshly than all the other reviews? 
5. Reach Goal: Look at the artists with at least one album review from 1999-2009 and at least one album review from 2014-2017. Of the top (three) highest-rated artists, is their recent album scored better than the average 2014-2017 album?

## Data Understanding 

1. Are self-released albums scored lower than the wider population? 

$H_0=$ Self-released albums are scored the same as or higher than the wider population.

$H_a=$ Self-released albums are scored lower than the wider population.

$p$_$value=.025$  

2. Are "Best New Music" albums more prevalent in a certain genre? MACARONI/GENRE N 

$H_0=$ BNM albums equally prevalent across genres.

$H_a=$ BNM albums are more prevalent in a one or more genres.

false positive = we say BNM albums are more prevalent in one or more genres, but they aren't
false negative = we say BNM albums are not more prevalent in one or more genres but they are

$p$_$value=.1$ (we're comfortable with being wrong about them being more prevalent in a certain genre one out of 10 times  - partially because if there is even a small chance that we see this pattern, we should take advantage of it, but if it doesn't actually exist it's no harm no foul - it's more okay to be wrong? )

3. Of the highest-scored labels, does on label have more "best new music" albums than the others?

$H_0=$ Of the highest-scored labels, BNM albums are equally prevalent across labels.

$H_a=$ Of the hightest-scored labels, BNM albums are more prevalent in one or more labels.

$p$_$value=.1$ (we're comfortable with being wrong about them being more prevalent across labels one out of 10 times  - partially because if there is even a small chance that we see this pattern, we should take advantage of it, but if it doesn't actually exist it's no harm no foul - it's more okay to be wrong? )

4. Does the reviewer giving the lowest average scores review more harshly than the average review score?

$H_0=$ X reviewer's scores are not much different than all the other scores. 

$H_a=$ X reviewer's scores are much different than all the other scores.

$p$_$value=.2$ (we're comfortable with being wrong about the reviewers scores being much different one out of 5 times  - partially because if there is even a small chance that you should ignore this one reviewer, we should take advantage of it, but if it doesn't actually exist it's not like we're firing that reviewer - we are just ignoring them needlessly) 

5. Reach Goal: Look at the artists with at least one album review from 1999-2009 and at least one album review from 2014-2017. Of the top (three) highest-rated (from 1999-2009) artists, is their recent album scored better than the average 2014-2017 album?

$H_0=$ The artists in this subset do not have scores that are higher than the modern average. 

$H_a=$ An artist(s) in this subset have higher scores than the modern average. MACARONI/3

false_positive = we say these artists have higher scores than the modern average, but they don't 
false_negative = we say these artists do not have higher scores than the modern average, but they do 

$p$_$value=.01$ (we're comfortable with being wrong about this 1 out of 100 times because we don't want to have a false positive - we don't want to waste time on artists that wer're not very confident are going to be successful (false positives)

## Data Preparation

We used pgloader to create our database and used SQL to query the database. 

1. Are self-released albums scored lower than the wider population? 

$H_0=$ Self-released albums are scored the same as or higher than the wider population.

$H_a=$ Self-released albums are scored lower than the wider population.

false positive = type 1 error =  we say sr are scored lower, but they aren't 

false negative = type 2 error = we say sr are not scored lower, but they are 

$p$_$value=.025$ (we're comfortable with one out of forty tests where we say they are lower, but they aren't, we want to be pretty sure that we're getting this right because we're telling her to ignore a whole "label" - if we say "they aren't scored lower and they are, then they are still getting ignored and that's okay) 

One-sample test makes sense here: 
- We assume the mean of the Pitchfork data is a close enough approximation of the population mean. 
- We don't have a control group and a experiment group - we have access to the "total population" and don't "need" a control group to represent that other part of the population. 


4. Does the reviewer giving the lowest average scores review more harshly than the average review score?

$H_0=$ X reviewer's scores are not much different than all the other scores. 

$H_a=$ X reviewer's scores are much different than all the other scores.

$p$_$value=.2$ (we're comfortable with being wrong about the reviewers scores being much different one out of 5 times  - partially because if there is even a small chance that you should ignore this one reviewer, we should take advantage of it, but if it doesn't actually exist it's not like we're firing that reviewer - we are just ignoring them needlessly) 


## Modeling

Pat's jn


## Evaluation

## Deployment 
(be sure to frame our limitations and what no-relationship means) 
